In [10]:
import io
import requests
import pandas as pd
import numpy as np
import OleFileIO_PL
import yaml

In [15]:
# Load Settings
with open("cepea.yaml", 'r') as stream:
    cepea = yaml.safe_load(stream)
cepea

{'cafe': 23, 'soja': 92, 'milho': 77, 'boi-gordo': 2}

## Importação dos dados

In [17]:
for k, v in cepea.items():

    # Cria e seta um dicionario
    dct = {}
    dct["codigo"] = k
    dct["id"] = v

    url = f'https://www.cepea.esalq.usp.br/br/indicador/series/{dct["codigo"]}.aspx?id={dct["id"]}'
    s=requests.get(url).content

    # Cria e organiza o Dataframe
    ole = OleFileIO_PL.OleFileIO(s)
    df = pd.read_excel(ole.openstream('Workbook'), skiprows=3)
    df = df.set_index("Data")
    df.index = df.index.rename('data')
    df.columns = ['preco_real','preco_dolar']
    
    # Checa Faltantes
    print(df.isnull().values.any())

    # Limpa zerados
    df = df[(df.T!=0).any()]
    
    # Salva os dados
    df.to_pickle(f'../dados/cepea-{dct["codigo"]}.p')

cafe
soja
milho
boi-gordo
